# SIMESH DEMO: INTRODUCTION AND MAIN FUNCS DISPLAY

## Data Preparations 

To reduce the size of the module, we do not include the datfile for display, but make a new AMRDataSet directly here with the TDm model constructed by the RBSL model (Titov et al, 2014; Titov et al. 2020).

In [1]:
# !!! It may take a while to initiate the TDm model for a large domain !!!

import numpy as np
import matplotlib.pyplot as plt
from simesh import configurations, load_from_uarrays, amr_loader

# prepare necessary parameters for initialization of a new AMRDataSet
xmin = np.array([-4,-4,0])
xmax = np.array([4,4,8])
domain_nx = np.array([30,30,30])
block_nx = np.array([10,10,10])

# parameters for the TDm model
r0 = 1.5
a0 = 0.3
q0 = 1.0
L0 = 1.0
d0 = 0.5
naxis = 500
ispositive = True

# prepare the mesh data of domain size
rho = np.ones(domain_nx) # uniform density just for display, no physical meaning
v1 = np.zeros(domain_nx)
v2 = np.zeros(domain_nx)
v3 = np.zeros(domain_nx)
TDm_slab = configurations.TDm_slab(xmin, xmax, domain_nx, r0, a0, ispositive, naxis, q0, L0, d0) # magnetic field b1,b2,b3 from TDm model

w_arrays = np.stack([rho, v1, v2, v3, TDm_slab[2], TDm_slab[1], TDm_slab[0]], axis=-1)

w_names = ['rho', 'v1', 'v2', 'v3', 'b1', 'b2', 'b3']

# here you can specify a datfile name for further output like: file_path='example.dat'; and any other parameters by **kwargs to update the default header
# you can view the default header from: simesh.header_template
ds = load_from_uarrays(w_arrays, w_names, xmin, xmax, block_nx, file_path='data/tdm.dat') 

## Interfaces of the AMRDataSet

#### I/O of AMRDataSet and Basic Attributes

If you here prefer to use the datfile from AMRVAC, you can use the following interfaces:

In [2]:
# first output the datfile to prepare the datfile (same as the datfile from AMRVAC)
ds.write_datfile()

# then read from the datfile just for display, same for those from AMRVAC
ds = amr_loader('data/tdm.dat')

assert np.all(ds.mesh.udata == w_arrays), "The data read from the datfile is not consistent with the data used to initialize the AMRDataSet"

the header is a dictionary, it is same as the header in the datfile containing the crucial information of the mesh and simulation parameters:

In [3]:
ds.header

{'datfile_version': 5,
 'offset_tree': 320,
 'offset_blocks': 1076,
 'nw': 7,
 'ndir': 3,
 'ndim': 3,
 'levmax': 1,
 'nleafs': 27,
 'nparents': 0,
 'it': 0,
 'time': 0.0,
 'xmin': array([-4., -4.,  0.]),
 'xmax': array([4., 4., 8.]),
 'domain_nx': array([30, 30, 30]),
 'block_nx': array([10, 10, 10]),
 'periodic': array([False, False, False]),
 'geometry': 'Cartesian_3D',
 'staggered': False,
 'w_names': ['rho', 'v1', 'v2', 'v3', 'b1', 'b2', 'b3'],
 'physics_type': 'mhd',
 'n_par': 1,
 'params': array([1.66666667]),
 'param_names': ['gamma'],
 'snapshotnext': 1,
 'slicenext': 0,
 'collapsenext': 0}

since it is a slab domain, we preserve the slab version of the mesh data with the shape same as the input data:

In [4]:
ds.mesh.udata.shape

(30, 30, 30, 7)

for the AMR structured mesh data, it is stored ordered by the Morton order with shape:

In [5]:
ds.mesh.data.shape

(27, 14, 14, 14, 7)

in which 27 is the number of blocks (3x3x3 here), (14,14,14) is (10,10,10) (block_nx) + 2 (nghostcells) * 2 (both sides), 7 is the number of physical variables

we can get the information (indices; level; morton_number[=sfc_index+1]; is_leaf) of each block in the 27 blocks by the sfc index [0-26]:

Here the indices [ig] is the index of the block in 3x3x3 level1 blocks array

In [6]:
ds.mesh.forest.sfc_to_node[2].node # note that ig is derived from node.ig1, node.ig2, node.ig3 

OctreeNode(ig=(0,1,0), level=1, morton=3, leaf=True)

In [7]:
# rnode is the attributes of each block
# xmin, ymin, zmin, xmax, ymax, zmax, dx, dy, dz for each block
ds.mesh.rnode[:,0]

array([-4.        , -4.        ,  0.        , -1.33333333, -1.33333333,
        2.66666667,  0.26666667,  0.26666667,  0.26666667])

### Ghostcells Filling and Manipulation over Blocks

In [8]:
# it may take minutes for enourmous blocks to fill the blocks
# only three-dimensional, Cartesian and constant physical boundary conditions are supported for now
ds.mesh.getbc()

With ghostcells, we can do interpolation and derivative jobs over the mesh by iterating the blocks, but for now, we only give how to output the uniform mesh. Other applications remain to be implemented. It is done by iterating ileafs blocks in ds.mesh.rnode[:,:nleafs]

In [9]:
xmin = [-2,-2,0]; xmax = [2,2,4]; nx = ny = nz = 100
udata = ds.mesh.export_slab_uniform(xmin, xmax, nx, ny, nz)

The current can be also derived in block-level with ghostcells:

In [10]:
ds.mesh.load_current()
ucurrent = ds.mesh.export_uniform_current(xmin, xmax, nx, ny, nz)